# Load modules

In [1]:
import os

from tqdm.auto import tqdm
from wgan_gp.models import Generator as WGAN_GENERATOR
from torch import nn
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
import numpy as np
import torch
import random
import torch.nn.functional as F
import torchvision
import pickle
import gc
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms
from diffusers import DDPMPipeline

In [2]:
device = torch.device("cuda:0")

# Diffusion models

In [3]:
models_path = [file for file in os.listdir("diffusion_models") if file != "outputs"]
batch_size = 100
models_path

['cosine', 'linear', 'sigmoid']

In [4]:
for model_name in models_path:
    model = DDPMPipeline.from_pretrained(os.path.join("diffusion_models", model_name)).to(device)
    images_path = os.path.join("generated_images", model_name)
    os.makedirs(images_path, exist_ok=True)
    for i in range(10000 // batch_size):
        images = model(num_inference_steps=100, batch_size=batch_size).images
        for j, image in enumerate(images):
            image.save(os.path.join(images_path, f"{i * batch_size + j}.png"))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
del model

In [6]:
gc.collect()

# GANs

## WGAN-GP

In [3]:
wgan_model = "WGAN-GP_512"
wgan_path = os.path.join("GANs", wgan_model)

In [4]:
generator = WGAN_GENERATOR(img_size=(64, 64, 3), latent_dim=512, dim=64).cuda()
generator.load_state_dict(torch.load(wgan_path))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Piotrek\AppData\Local\Temp\ipykernel_59368\2015595999.py:2 in <module>                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Piotrek\\AppData\\Local\\Temp\\ipykernel_59368\\2015595999.py'                       │
│                                                                                                  │
│ C:\Users\Piotrek\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:791 in load     │
│                                                                                                  │
│    788 │   if 'encoding' not in pickle_load_args.keys():                                         │
│    789 │   │   pickle_load_args['encoding'] = 'utf-8'                                            │
│    790 │                                                                                         │
│ ❱  791 │   with _open_file_like(f, 'rb') as opened_file:                                         │
│    792 │   │   if _is_zipfile(opened_file):                                                      │
│    793 │   │   │   # The zipfile reader is going to advance the current file position.           │
│    794 │   │   │   # If we want to actually tail call to torch.jit.load, we need to              │
│                                                                                                  │
│ C:\Users\Piotrek\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:271 in          │
│ _open_file_like                                                                                  │
│                                                                                                  │
│    268                                                                                           │
│    269 def _open_file_like(name_or_buffer, mode):                                                │
│    270 │   if _is_path(name_or_buffer):                                                          │
│ ❱  271 │   │   return _open_file(name_or_buffer, mode)                                           │
│    272 │   else:                                                                                 │
│    273 │   │   if 'w' in mode:                                                                   │
│    274 │   │   │   return _open_buffer_writer(name_or_buffer)                                    │
│                                                                                                  │
│ C:\Users\Piotrek\anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:252 in __init__ │
│                                                                                                  │
│    249                                                                                           │
│    250 class _open_file(_opener):                                                                │
│    251 │   def __init__(self, name, mode):                                                       │
│ ❱  252 │   │   super().__init__(open(name, mode))                                                │
│    253 │                                                                                         │
│    254 │   def __exit__(self, *args):                                                            │
│    255 │   │   self.file_like.close()                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
PermissionError: [Errno 13] Permission denied: 'GANs\\WGAN-GP_512'

In [5]:
batch_size = 100
images_path = os.path.join("generated_images", wgan_model)
os.makedirs(images_path, exist_ok=True)
for i in range(10000 // batch_size):
    rand = generator.sample_latent(batch_size).to(device)
    output = generator(rand)
    for j in range(batch_size):
        image = transforms.ToPILImage()(output[j, ...])
        image.save(os.path.join(images_path, f"{i * batch_size + j}.png"))

## DC GANs

In [6]:
GENERATOR_FEATURES_SIZE = 64
IMG_CHANNELS = 3

class Generator(nn.Module):
    def __init__(self, latent_space_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(latent_space_dim, GENERATOR_FEATURES_SIZE * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(GENERATOR_FEATURES_SIZE * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(GENERATOR_FEATURES_SIZE * 8, GENERATOR_FEATURES_SIZE * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(GENERATOR_FEATURES_SIZE * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( GENERATOR_FEATURES_SIZE * 4, GENERATOR_FEATURES_SIZE * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(GENERATOR_FEATURES_SIZE * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( GENERATOR_FEATURES_SIZE * 2, GENERATOR_FEATURES_SIZE, 4, 2, 1, bias=False),
            nn.BatchNorm2d(GENERATOR_FEATURES_SIZE),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( GENERATOR_FEATURES_SIZE, IMG_CHANNELS, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

In [7]:
batch_size = 100
dc_gans = [file for file in os.listdir("GANs") if file.startswith("DC-GAN")]
for dc_gan in dc_gans:
    latent_dim = int(dc_gan.split("_")[-1])
    netG = Generator(latent_dim).to(device)
    netG.load_state_dict(torch.load(os.path.join("GANs", dc_gan, "generator")))
    images_path = os.path.join("generated_images", dc_gan)
    os.makedirs(images_path, exist_ok=True)
    for i in range(10000 // batch_size):
        fixed_noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        output = (netG(fixed_noise) + 1) / 2
        for j in range(batch_size):
            image = transforms.ToPILImage()(output[j, ...])
            image.save(os.path.join(images_path, f"{i * batch_size + j}.png"))